<a href="https://colab.research.google.com/github/lkarjun/malayalam-language-model/blob/language-model/training_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset Loading

In [7]:
!pip install -qq dvc[gdrive]

!dvc get https://github.com/lkarjun/malayalam-language-model \
Datasets/


    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Enter verification code: 4/1AX4XfWiN4jIYvY9yEstRPTwg940SXCddQl9Gj5SsZRLcKsq4IVA0yjkzxGI
Authentication successful.


In [8]:
!unzip -q 'Datasets/*.zip' -d Datasets/


3 archives were successfully processed.


In [13]:
import pandas as pd
from pathlib import Path

DS_Path = Path("/content/Datasets")

csv_files = ["magazine_files.csv", 
             "wikitext_files.csv",
             "article_files.csv"
             ]
             
df = pd.concat([pd.read_csv(DS_Path/csv) for csv in csv_files])

In [41]:
sample = df['file_path'][:10].to_list()

with open(sample[0], "r") as file:
  sample_txt = file.read()

### Imports

In [1]:
!pip install -qq tokenizer transformers

     |████████████████████████████████| 77 kB 2.6 MB/s 
     |████████████████████████████████| 3.5 MB 9.2 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
     |████████████████████████████████| 6.8 MB 37.0 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 


In [237]:
from tokenizers import Tokenizer, pre_tokenizers, decoders
from tokenizers.models import Unigram, WordPiece
from tokenizers.trainers import UnigramTrainer, WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.pre_tokenizers import Digits
from tokenizers.normalizers import Strip

## Training Subword Tokenizer For Malayalm 

In [259]:
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Strip()
tokenizer.decoders = decoders.WordPiece()

trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [258]:
pre_tokenizer = pre_tokenizers.Sequence([
                                         Whitespace(),  
                                         Digits(individual_digits=False)
                                        ])


tokenizer.pre_tokenizer = pre_tokenizer

# training tokenizer
tokenizer.train(sample, trainer)

In [243]:
tokenizer.post_processor = TemplateProcessing(
                                single="[CLS] $A [SEP]",
                                pair="[CLS] $A [SEP] $B:1 [SEP]:1",
                                special_tokens=[
                                        ("[CLS]", tokenizer.token_to_id("[CLS]")),
                                        ("[SEP]", tokenizer.token_to_id("[SEP]")),
                                  ],
                              )

In [278]:
output = tokenizer.encode(sample_txt)
print(output.tokens)

['[CLS]', 'ഇന്ത്യയിലെ', 'ആദ്യത്തെ', 'വനിതാ', 'ഐ\u200c', '##എ', '##\u200cഎസ്', 'ഉദ്യോഗസ്ഥ', '##യെ', 'കുറിച്ച', '##റിയാമോ', '?', 'അവരുടെ', 'പേരാണ്', 'അന്ന', 'രാജ', '##ം', 'മൽഹോത്ര', '.', 'അവർ', 'ഒരു', 'മലയാളി', 'കൂടിയാണ്', 'എന്നത്', 'നാം', 'മലയാളികൾക്ക്', 'അഭിമാന', '##ിക്കാനുള്ള', 'വകയാണ്', '.', 'കുഞ്ഞുകുട്ട', '##ി', 'പ്രാര', '##ാബ്ദ', '##ങ്ങളുമായി', 'വീടുകളിൽ', 'ഒതുങ്ങി', 'കഴിഞ്ഞിരുന്ന', 'സ്ത്രീകൾ', '##ക്കിടയിൽ', 'അന്ന', 'രാജ', '##ം', 'മൽ', '##\u200c', '##ഹോ', '##ത്ര', 'വേറിട്ട്', 'നിന്നു', '.', 'സമൂഹത്തിൽ', 'നിലനിന്നിരുന്ന', 'അസമത്വ', '##ത്തിനെതിരെ', ',', 'ലിംഗ', 'അനീതി', '##കൾക്ക', '##െ', '##തിരെ', 'പോരാടിയ', 'സ്ത്രീയാണ്', 'അവർ', ',', 'ഇന്ത്യയിലെ', 'ആദ്യത്തെ', 'വനിതാ', 'ഐ\u200c', '##എ', '##\u200cഎസ്', 'ഉദ്യോഗസ്ഥ', '.', 'കേന്ദ്രസർക്കാര', '##ിൽ', 'സെക്രട്ടേറിയ', '##ൽ', 'പദവി', 'വഹിച്ച', 'ആദ്യ', 'വനിത', 'കൂടിയായിരുന്നു', 'അവർ', '.', 'വളരെ', 'പ്രചോദന', '##ാത്മകമായ', 'അവരുടെ', 'കഥ', '.', '1927', 'ജൂലൈ', '17', '-', 'ന്', 'കേരളത്തിലെ', 'നിരണം', 'ഗ്രാമത്തിൽ', 'ജനിച്ച', 'അന്ന', ',', 'മദ്രാസ്',

In [279]:
print(output.ids)

[1, 4119, 3785, 8630, 81674, 1336, 38131, 5036, 2309, 9826, 34449, 36, 2721, 7114, 3182, 2355, 1286, 54142, 19, 2570, 2111, 7422, 5176, 3711, 4703, 34496, 7682, 3579, 98572, 19, 62829, 1270, 36210, 11545, 4822, 10428, 17338, 22177, 7310, 5570, 3182, 2355, 1286, 12164, 1305, 19509, 2093, 34326, 2947, 19, 11491, 14297, 54015, 10160, 17, 10152, 60904, 40243, 1276, 3674, 25485, 30493, 2570, 17, 4119, 3785, 8630, 81674, 1336, 38131, 5036, 19, 37784, 2073, 34963, 1303, 10757, 18426, 2406, 8832, 12882, 2570, 19, 2745, 13851, 15076, 2721, 2447, 19, 25930, 6094, 4209, 18, 2896, 3991, 65621, 8841, 7316, 3182, 17, 11240, 14049, 2448, 5292, 19807, 16340, 9973, 5288, 19, 11559, 18, 4970, 12905, 8752, 48719, 2893, 2461, 3182, 20150, 19, 7891, 4877, 3906, 2111, 9986, 12905, 8752, 6815, 8773, 73904, 1305, 7326, 25789, 2950, 14345, 9004, 19, 11180, 29434, 544, 47724, 6449, 28998, 19, 2593, 17, 74143, 13056, 2390, 3182, 18391, 4355, 19, 4518, 4679, 2079, 2136, 35507, 2338, 6449, 17031, 24079, 19, 2593, 

In [280]:
tokenizer.decode(output.ids, skip_special_tokens=True)

"ഇന്ത്യയിലെ ആദ്യത്തെ വനിതാ ഐ\u200c ##എ ##\u200cഎസ് ഉദ്യോഗസ്ഥ ##യെ കുറിച്ച ##റിയാമോ ? അവരുടെ പേരാണ് അന്ന രാജ ##ം മൽഹോത്ര . അവർ ഒരു മലയാളി കൂടിയാണ് എന്നത് നാം മലയാളികൾക്ക് അഭിമാന ##ിക്കാനുള്ള വകയാണ് . കുഞ്ഞുകുട്ട ##ി പ്രാര ##ാബ്ദ ##ങ്ങളുമായി വീടുകളിൽ ഒതുങ്ങി കഴിഞ്ഞിരുന്ന സ്ത്രീകൾ ##ക്കിടയിൽ അന്ന രാജ ##ം മൽ ##\u200c ##ഹോ ##ത്ര വേറിട്ട് നിന്നു . സമൂഹത്തിൽ നിലനിന്നിരുന്ന അസമത്വ ##ത്തിനെതിരെ , ലിംഗ അനീതി ##കൾക്ക ##െ ##തിരെ പോരാടിയ സ്ത്രീയാണ് അവർ , ഇന്ത്യയിലെ ആദ്യത്തെ വനിതാ ഐ\u200c ##എ ##\u200cഎസ് ഉദ്യോഗസ്ഥ . കേന്ദ്രസർക്കാര ##ിൽ സെക്രട്ടേറിയ ##ൽ പദവി വഹിച്ച ആദ്യ വനിത കൂടിയായിരുന്നു അവർ . വളരെ പ്രചോദന ##ാത്മകമായ അവരുടെ കഥ . 1927 ജൂലൈ 17 - ന് കേരളത്തിലെ നിരണം ഗ്രാമത്തിൽ ജനിച്ച അന്ന , മദ്രാസ് സർവകലാശാലയിൽ നിന്ന് ഇംഗ്ലീഷ് സാഹിത്യത്തിൽ ബിരുദാനന്തര ബിരുദം നേടി . 1950 - ലാണ് സിവിൽ സർവീസ് പരീക്ഷയ്ക്ക് ശ്രമ ##ിക്കാൻ അന്ന തീരുമാനിക്കുന്നത് . ഒരുപക്ഷേ അന്നത്തെ ##ക്കാലത്ത് ഒരു പെൺകുട്ടി സിവിൽ സർവീസ് സ്വപ്നം കാണുക ##യെന്നത് ##\u200c ആർക്കും ചിന്തിക്കാൻ പോലും സാധിക്കാത്ത കാര്യമാണ് . തീർത്തും വിപ്ലവകരമായ ആ 

In [255]:
check_unk = tokenizer.encode("Hello")
check_unk.tokens, check_unk.ids, tokenizer.id_to_token(0)

(['[CLS]', '[UNK]', '[SEP]'], [1, 0, 2], '[UNK]')

### Training Tokenizer

In [270]:
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Strip()
tokenizer.decoders = decoders.WordPiece()

trainer = WordPieceTrainer( vocab_size=100000,
                            special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [271]:
pre_tokenizer = pre_tokenizers.Sequence([
                                         Whitespace(),  
                                         Digits(individual_digits=False)
                                        ])


tokenizer.pre_tokenizer = pre_tokenizer

# training tokenizer
tokenizer.train(df['file_path'], trainer)

In [272]:
tokenizer.post_processor = TemplateProcessing(
                                single="[CLS] $A [SEP]",
                                pair="[CLS] $A [SEP] $B:1 [SEP]:1",
                                special_tokens=[
                                        ("[CLS]", tokenizer.token_to_id("[CLS]")),
                                        ("[SEP]", tokenizer.token_to_id("[SEP]")),
                                  ],
                              )

In [273]:
tokenizer.enable_padding(pad_id=3, pad_token="[PAD]")

In [274]:
tokenizer.save("tokenizer-malayalam.json")

In [275]:
tokenizer.get_vocab_size()

100000